In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
com1 = yf.Ticker("PEP")
data1 = com1.history(period = "35y")
com2 = yf.Ticker("AAPL")
data2 = com2.history(period = "35y")
com3 = yf.Ticker("TXN")
data3 = com3.history(period = "35y")
com4 = yf.Ticker("MSFT")
data4 = com4.history(period = "35y")
com5 = yf.Ticker("ADP")
data5 = com5.history(period = "35y")
com6 = yf.Ticker("INTC")
data6 = com6.history(period = "35y")

In [ ]:
print(1,len(data1))
print(2,len(data2))
print(3,len(data3))
print(4,len(data4))
print(5,len(data5))
print(6,len(data6))

1 8819
2 8819
3 8819
4 8819
5 8819
6 8819


In [ ]:
def calculate_technical_indicators(data):
    
    data = data.iloc[:,:5]
    data = data.reset_index(drop = False)

    # Calculate the typical price
    data['Typical Price'] = (data['High'] + data['Low'] + data['Close']) / 3

    # Calculate the cumulative volume and cumulative TP*V up to each row
    data['Cumulative Volume'] = data['Volume'].cumsum()
    df2 = data['Typical Price'] * data['Volume']
    data['Cumulative TP*V'] = df2.cumsum()

    # Calculate the VWAP at each row
    data['VWAP'] = data['Cumulative TP*V'] / data['Cumulative Volume']

    # Print the dataframe with the VWAP column
    
    lag_features = ["Open","High", "Low","VWAP"]
    window1 = 3
    window2 = 7

    df_rolled_3d = data[lag_features].rolling(window=window1, min_periods=0)
    df_mean_3d = df_rolled_3d.mean().shift(1).reset_index().astype(np.float32)

    df_rolled_7d = data[lag_features].rolling(window=window2, min_periods=0)
    df_std_7d = df_rolled_7d.std().shift(1).reset_index().astype(np.float32)

    for feature in lag_features:
        data[f"{feature}_mean_lag{window1}"] = df_mean_3d[feature].astype('float32')
        data[f"{feature}_std_lag{window2}"] = df_std_7d[feature].astype('float32')
    
    data.fillna(data.mean(), inplace=True)
    data.set_index("Date", drop=False, inplace=True)
    #data = data.drop(["Typical Price","Cumulative Volume","Cumulative TP*V"],axis = 1)
    # Convert the date column to a datetime object
    
    data['Date'] = pd.to_datetime(data['Date'])

    # Create new features
    data['month'] = data['Date'].dt.month
    data['day'] = data['Date'].dt.day
    data['day_of_week'] = data['Date'].dt.dayofweek
    data['week_of_year'] = data['Date'].dt.isocalendar().week.astype(np.int64)
    # Calculate moving averages
    data['ma_5'] = data['Close'].rolling(5).mean()
    data['ma_10'] = data['Close'].rolling(10).mean()
    data['ma_20'] = data['Close'].rolling(20).mean()
    data['ma_50'] = data['Close'].rolling(50).mean()

    # Calculate the relative strength index (RSI)
    rsi_period = 14
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['rsi'] = rsi

    # Calculate the rate of change (ROC)
    roc_period = 12
    roc = (data['Close'] / data['Close'].shift(roc_period) - 1) * 100
    data['roc'] = roc

    # Calculate the on-balance volume (OBV)
    obv = np.zeros(len(data))
    obv[0] = data['Volume'][0]
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1]:
            obv[i] = obv[i-1] + data['Volume'][i]
        elif data['Close'][i] < data['Close'][i-1]:
            obv[i] = obv[i-1] - data['Volume'][i]
        else:
            obv[i] = obv[i-1]
    data['obv'] = obv

    # Calculate the relative volatility index (RVI)
    rvi_period = 10
    delta_close = data['Close'].diff()
    delta_open = data['Open'].diff()
    rvi = np.zeros(len(data))
    for i in range(1, len(data)):
        if delta_close[i] > delta_open[i]:
            rvi[i] = ((delta_close[i] / (data['High'][i] - data['Low'][i])) + 
                      (delta_close[i-1] / (data['High'][i-1] - data['Low'][i-1]))) / 2 * 100
        elif delta_close[i] < delta_open[i]:
            rvi[i] = ((delta_close[i] / (data['High'][i] - data['Low'][i])) + 
                      (delta_close[i-1] / (data['High'][i-1] - data['Low'][i-1]))) / 2 * 100
        else:
            rvi[i] = ((delta_open[i] / (data['High'][i] - data['Low'][i])) + 
                      (delta_open[i-1] / (data['High'][i-1] - data['Low'][i-1]))) / 2 * 100
    data['rvi'] = rvi

    # Calculate the stochastic oscillator
    k_period = 14
    d_period = 3
    low_min = data['Low'].rolling(k_period).min()
    high_max = data['High'].rolling(k_period).max()
    k_percent = ((data['Close'] - low_min) / (high_max - low_min)) * 100
    d_percent = k_percent.rolling(d_period).mean()
    data['k_percent'] = k_percent
    data['d_percent'] = d_percent
    # Smooth the stochastic oscillator using an exponential moving average (EMA)
    d_percent_smoothed = d_percent.ewm(span=d_period, min_periods=d_period).mean()
    #data['stochastic_k'] = k_percent
    data['stochastic_d'] = d_percent_smoothed

    # Calculate the Chaikin Money Flow (CMF)
    close_location_value = ((data['Close'] - data['Low']) - (data['High'] - data['Close'])) / (data['High'] - data['Low'])
    money_flow_volume = close_location_value * data['Volume']
    adl = np.zeros(len(data))
    adl[0] = money_flow_volume[0]
    adl = pd.Series(adl)
    
    for i in range(1, len(data)):
      adl[i] = adl[i-1] + money_flow_volume[i]
    
    adl_ma = adl.rolling(window=20).mean()
    adl_ma_vol = data['Volume'].rolling(window=20).mean()
    adl_ma = adl_ma.fillna(1)
    adl_ma_vol = adl_ma_vol.fillna(1)
    cmf = adl_ma.values/adl_ma_vol.values
    data['cmf'] = cmf

    # Calculate the Accumulation/Distribution Line (ADL)

    adl = np.zeros(len(data))
    adl[0] = ((data['Close'][0] - data['Low'][0]) - (data['High'][0] - data['Close'][0])) / (data['High'][0] - data['Low'][0]) * data['Volume'][0]
    for i in range(1, len(data)):
        clv = ((data['Close'][i] - data['Low'][i]) - (data['High'][i] - data['Close'][i])) / (data['High'][i] - data['Low'][i])
        adl[i] = adl[i-1] + clv * data['Volume'][i]
    data['adl'] = adl
    
    
    
    
    data.dropna(inplace=True)
    features = ['Date','Open','Close','High','Low','Typical Price','Volume','VWAP','adl','cmf',
                'stochastic_d','d_percent','k_percent','rvi','obv','roc',
                'rsi','ma_50','ma_20','ma_10','ma_5','week_of_year','day_of_week',
                'day','month','Open_mean_lag3','Open_std_lag7', 'High_mean_lag3', 
                'High_std_lag7', 'Low_mean_lag3','Low_std_lag7', 'VWAP_mean_lag3', 'VWAP_std_lag7']
    
    data = data[features]
    return data




In [ ]:
print(1,len(data1))
print(2,len(data2))
print(3,len(data3))
print(4,len(data4))
print(5,len(data5))
print(6,len(data6))

1 8770
2 8770
3 8770
4 8770
5 8770
6 8770


In [ ]:
data1 = calculate_technical_indicators(data1)
data2 = calculate_technical_indicators(data2)
data3 = calculate_technical_indicators(data3)
data4 = calculate_technical_indicators(data4)
data5 = calculate_technical_indicators(data5)
data6 = calculate_technical_indicators(data6)

<ipython-input-7-36b9a8baa4dd>:33: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  data.fillna(data.mean(), inplace=True)
<ipython-input-7-36b9a8baa4dd>:33: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  data.fillna(data.mean(), inplace=True)
<ipython-input-7-36b9a8baa4dd>:33: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  data.fillna(data.mean(), inplace=True)
<ipython-input-7-36b9a8baa4dd>:33: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  data.fillna(data.mean(), inplace=True)
<ipython-input-7-36b9a8baa4dd>:33: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will includ

In [ ]:
data1.pop('Date')
data2.pop('Date')
data3.pop('Date')
data4.pop('Date')
data5.pop('Date')
data6.pop('Date')

Date
1988-07-06 00:00:00-04:00   1988-07-06 00:00:00-04:00
1988-07-07 00:00:00-04:00   1988-07-07 00:00:00-04:00
1988-07-08 00:00:00-04:00   1988-07-08 00:00:00-04:00
1988-07-11 00:00:00-04:00   1988-07-11 00:00:00-04:00
1988-07-12 00:00:00-04:00   1988-07-12 00:00:00-04:00
                                       ...           
2023-04-19 00:00:00-04:00   2023-04-19 00:00:00-04:00
2023-04-20 00:00:00-04:00   2023-04-20 00:00:00-04:00
2023-04-21 00:00:00-04:00   2023-04-21 00:00:00-04:00
2023-04-24 00:00:00-04:00   2023-04-24 00:00:00-04:00
2023-04-25 00:00:00-04:00   2023-04-25 00:00:00-04:00
Name: Date, Length: 8770, dtype: datetime64[ns, America/New_York]

In [ ]:
data1

,Open,Close,High,Low,Typical Price,Volume,VWAP,adl,cmf,stochastic_d,...,day,month,Open_mean_lag3,Open_std_lag7,High_mean_lag3,High_std_lag7,Low_mean_lag3,Low_std_lag7,VWAP_mean_lag3,VWAP_std_lag7
Date,,,,,,,,,,,,,,,,,,,,,
1988-07-06 00:00:00-04:00,2.668166,2.578025,2.668166,2.559997,2.602063,3496200,2.575902,3.070060e+07,8.375036,54.266271,...,6,7,2.668166,0.021904,2.689199,0.014455,2.644129,0.012125,2.574306,0.003209
1988-07-07 00:00:00-04:00,2.587037,2.596051,2.614079,2.550981,2.587037,2701800,2.576082,3.185851e+07,8.957496,45.488196,...,7,7,2.668166,0.020818,2.680185,0.017148,2.611077,0.032441,2.575223,0.002922
1988-07-08 00:00:00-04:00,2.578024,2.587039,2.605066,2.569010,2.587038,2035200,2.576214,3.185853e+07,9.255355,33.038223,...,8,7,2.632109,0.035243,2.650137,0.033613,2.584034,0.043808,2.575773,0.002407
1988-07-11 00:00:00-04:00,2.587037,2.596051,2.614079,2.569009,2.593047,1646400,2.576376,3.218781e+07,9.630924,29.482053,...,11,7,2.611076,0.045626,2.629104,0.042736,2.559996,0.046798,2.576066,0.001750
1988-07-12 00:00:00-04:00,2.605067,2.578025,2.605067,2.559997,2.581030,3092400,2.576459,3.156931e+07,9.652157,25.852353,...,12,7,2.584033,0.047699,2.611075,0.041215,2.563000,0.044899,2.576224,0.001177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-19 00:00:00-04:00,185.240005,184.720001,185.320007,184.259995,184.766668,2209900,50.845929,2.028128e+09,576.672270,87.854964,...,19,4,184.100006,0.779092,184.663330,0.552221,183.380005,0.743005,50.829971,0.023364
2023-04-20 00:00:00-04:00,184.839996,185.330002,186.039993,184.199997,185.189997,3868200,50.858551,2.029011e+09,581.016210,88.678155,...,20,4,184.733337,1.010906,185.096664,0.632151,183.826660,0.790102,50.838493,0.021567
2023-04-21 00:00:00-04:00,185.990005,185.410004,186.380005,185.130005,185.640004,3781800,50.870931,2.026924e+09,588.496546,87.629125,...,21,4,184.949997,1.073514,185.526672,0.781983,184.066666,0.877286,50.847740,0.020693


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Portfolio Optimization/Pepsico.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Portfolio Optimization/Apple.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Portfolio Optimization/Netflix.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Portfolio Optimization/Tesla.csv')
df5 = pd.read_csv('/content/drive/MyDrive/Portfolio Optimization/Starbucks.csv')
df6 = pd.read_csv('/content/drive/MyDrive/Portfolio Optimization/Intel.csv')

In [ ]:
def stringdate(row):
  return str(row.month) + '/' + str(row.year)

In [ ]:
def news_features(data1,df1):
  cnn = df1[df1['Series'] == 'CNN']
  fox = df1[df1['Series'] == 'FOXNEWS']
  fox = fox.reset_index(drop = True)
  msn = df1[df1['Series'] == 'MSNBC']
  msn = msn.reset_index(drop = True)
  msn.columns = [0,1,2]
  msn[3] = cnn['Value']
  msn[4] = fox['Value']
  msn = msn.drop(1,axis = 1)
  idx = msn[msn[0] == '4/2013'].index
  idx = idx[0]
  msn = msn.iloc[idx:,:]
  msn.columns = ['stringdate',0,1,2]
  data1['stringdate'] = data1['Date'].apply(stringdate)
  print(len(data1))
  data1 = pd.merge(left = data1,right = msn,on = 'stringdate')
  print(len(data1))
  data1 = data1.drop(['stringdate','Date'],axis = 1)
  return data1

In [ ]:
data1 = news_features(data1,df1)
data2 = news_features(data2,df2)
data3 = news_features(data3,df3)
data4 = news_features(data4,df4)
data5 = news_features(data5,df5)
data6 = news_features(data6,df6)

2971
2536
2971
2536
2971
2536
2971
2536
2971
2536
2971
2536


In [ ]:
data1

,Open,Close,High,Low,Typical Price,Volume,VWAP,adl,cmf,stochastic_d,...,Open_std_lag7,High_mean_lag3,High_std_lag7,Low_mean_lag3,Low_std_lag7,VWAP_mean_lag3,VWAP_std_lag7,0,1,2
0,59.114402,59.369591,59.467168,59.114402,59.317053,3639000,48.768103,1.740768e+08,30.278521,81.729679,...,0.724869,59.224483,0.761450,58.486431,0.668849,48.742203,0.044283,0.0,0.0,0.0
1,59.557241,59.849960,59.955039,59.497194,59.767398,4034100,48.781223,1.762592e+08,30.894902,88.581389,...,0.744370,59.297039,0.660388,58.729115,0.693224,48.755043,0.040066,0.0,0.0,0.0
2,59.947527,59.264511,59.955028,59.144424,59.454654,3895400,48.793502,1.735180e+08,32.084920,88.870901,...,0.618487,59.604767,0.320579,59.069366,0.628168,48.768684,0.030857,0.0,0.0,0.0
3,58.769151,59.692341,59.722364,58.769151,59.394618,5299000,48.810067,1.784832e+08,31.857019,88.378712,...,0.481902,59.792412,0.389274,59.252007,0.470347,48.780941,0.028586,0.0,0.0,0.0
4,59.257031,58.986828,59.354608,58.844225,59.061887,4244000,48.822880,1.766107e+08,32.056890,83.192858,...,0.500108,59.877476,0.352972,59.136925,0.408372,48.794930,0.028761,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,185.240005,184.720001,185.320007,184.259995,184.766668,2209900,93.957678,6.723623e+08,190.344781,87.854964,...,0.779092,184.663330,0.552221,183.380005,0.743005,93.928375,0.042782,0.0,0.0,0.0
2532,184.839996,185.330002,186.039993,184.199997,185.189997,3868200,93.980897,6.732453e+08,191.846496,88.678155,...,1.010906,185.096664,0.632151,183.826660,0.790102,93.944023,0.039521,0.0,0.0,0.0
2533,185.990005,185.410004,186.380005,185.130005,185.640004,3781800,94.003698,6.711578e+08,194.384222,87.629125,...,1.073514,185.526672,0.781983,184.066666,0.877286,93.961014,0.037966,0.0,0.0,0.0
2534,186.500000,185.500000,186.570007,185.020004,185.696671,3540200,94.025046,6.698102e+08,194.912813,85.752796,...,1.147112,185.913330,0.822119,184.529999,0.982725,93.980759,0.038146,0.0,0.0,0.0


In [ ]:
data1.to_csv('/content/drive/MyDrive/Portfolio Optimization/data7_more_years.csv',index = False)
data2.to_csv('/content/drive/MyDrive/Portfolio Optimization/data8_more_years.csv',index = False)
data3.to_csv('/content/drive/MyDrive/Portfolio Optimization/data9_more_years.csv',index = False)
data4.to_csv('/content/drive/MyDrive/Portfolio Optimization/data10_more_years.csv',index = False)
data5.to_csv('/content/drive/MyDrive/Portfolio Optimization/data11_more_years.csv',index = False)
data6.to_csv('/content/drive/MyDrive/Portfolio Optimization/data12_more_years.csv',index = False)

In [ ]:
data1

,Open,Close,High,Low,Typical Price,Volume,VWAP,adl,cmf,stochastic_d,...,day,month,Open_mean_lag3,Open_std_lag7,High_mean_lag3,High_std_lag7,Low_mean_lag3,Low_std_lag7,VWAP_mean_lag3,VWAP_std_lag7
Date,,,,,,,,,,,,,,,,,,,,,
1988-07-06 00:00:00-04:00,2.668166,2.578025,2.668166,2.559997,2.602063,3496200,2.575902,3.070060e+07,8.375036,54.266271,...,6,7,2.668166,0.021904,2.689199,0.014455,2.644129,0.012125,2.574306,0.003209
1988-07-07 00:00:00-04:00,2.587037,2.596051,2.614079,2.550981,2.587037,2701800,2.576082,3.185851e+07,8.957496,45.488196,...,7,7,2.668166,0.020818,2.680185,0.017148,2.611077,0.032441,2.575223,0.002922
1988-07-08 00:00:00-04:00,2.578024,2.587039,2.605066,2.569010,2.587038,2035200,2.576214,3.185853e+07,9.255355,33.038223,...,8,7,2.632109,0.035243,2.650137,0.033613,2.584034,0.043808,2.575773,0.002407
1988-07-11 00:00:00-04:00,2.587037,2.596051,2.614079,2.569009,2.593047,1646400,2.576376,3.218781e+07,9.630924,29.482053,...,11,7,2.611076,0.045626,2.629104,0.042736,2.559996,0.046798,2.576066,0.001750
1988-07-12 00:00:00-04:00,2.605067,2.578025,2.605067,2.559997,2.581030,3092400,2.576459,3.156931e+07,9.652157,25.852353,...,12,7,2.584033,0.047699,2.611075,0.041215,2.563000,0.044899,2.576224,0.001177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-19 00:00:00-04:00,185.240005,184.720001,185.320007,184.259995,184.766668,2209900,50.845929,2.028128e+09,576.672270,87.854964,...,19,4,184.100006,0.779092,184.663330,0.552221,183.380005,0.743005,50.829971,0.023364
2023-04-20 00:00:00-04:00,184.839996,185.330002,186.039993,184.199997,185.189997,3868200,50.858551,2.029011e+09,581.016210,88.678155,...,20,4,184.733337,1.010906,185.096664,0.632151,183.826660,0.790102,50.838493,0.021567
2023-04-21 00:00:00-04:00,185.990005,185.410004,186.380005,185.130005,185.640004,3781800,50.870931,2.026924e+09,588.496546,87.629125,...,21,4,184.949997,1.073514,185.526672,0.781983,184.066666,0.877286,50.847740,0.020693
